# Predicting House Prices

In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

57344/57026 [==============================] - 0s 3us/step


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

1. Per capita crime rate.(人均犯罪率)
2. Proportion of residential land zoned for lots over 25,000 square feet. (佔地超過25,000平方英尺的住宅用地比例)
3. Proportion of non-retail business acres per town. (每個城鎮非零售業務英畝的比例)
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). (查爾斯河虛擬變量（如果束縛河流，則為1；否則為0）)
5. Nitric oxides concentration (parts per 10 million).(一氧化氮濃度（百萬分之幾）)
6. Average number of rooms per dwelling. (每個住宅的平均房間數)
7. Proportion of owner-occupied units built prior to 1940. (1940年之前建造的自有住房的比例)
8. Weighted distances to five Boston employment centres. (到五個波士頓就業中心的加權距離)
9. Index of accessibility to radial highways. (徑向公路的可達性指數)
10. Full-value property-tax rate per $10,000. (每10,000美元的全值財產稅率)
11. Pupil-teacher ratio by town. (各鎮的師生比例)
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town. (1000 *（Bk-0.63）** 2其中Bk是按城鎮劃分的黑人比例)
13. % lower status of the population. (人口降低％)

In [4]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [5]:
from keras import models
from keras import layers

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [6]:
import numpy as np

k = 4    # 4-fold Cross Validation (四折交叉驗證)
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0






processing fold # 1
processing fold # 2
processing fold # 3


In [ ]:
all_scores